In [15]:
import pandas as pd
from collections import defaultdict
import csv

In [16]:
import pandas as pd

# Đọc dữ liệu từ file CSV
data = pd.read_csv('clean data.csv')

# Trộn các hàng trong dữ liệu
shuffled_data = data.sample(frac=1)

# Lưu dữ liệu trộn vào file CSV
shuffled_data.to_csv('file_shuffled.csv', index=False)


**Đọc file clean data** 

In [17]:
df = pd.read_csv('clean data.csv')

**Thêm các cột chênh lệch về rồng, sứ giả, trụ phút thứ 15 và chênh lệch về kda (kil - death - assist) giữa các đường**

In [18]:
df["herald_difference"] = df["blue_heralds"] - df["red_heralds"]
df["dragon_difference"] = df["dragon_blue"] - df["dragon_red"]
df["turret_difference"] = df["turret_blue"] - df["turret_red"]

df["kda_difference_top"] = df["kda_top_blue"] - df["kda_top_red"]
df["kda_difference_jungle"] = df["kda_jungle_blue"] - df["kda_jungle_red"]
df["kda_difference_mid"] = df["kda_mid_blue"] - df["kda_mid_red"]
df["kda_difference_ad"] = df["kda_ad_blue"] - df["kda_ad_red"]
df["kda_difference_sp"] = df["kda_sp_blue"] - df["kda_sp_red"]

**Xóa những cột dư thừa sau khi tính chênh lệch**

In [19]:
df = df.drop(['kda_top_blue', 'kda_top_red', 'kda_jungle_blue', 'kda_jungle_red', 'kda_mid_blue', 'kda_mid_red', 
              'kda_ad_blue', 'kda_ad_red', 'kda_sp_blue', 'kda_sp_red', 'link','dragon_blue', 'dragon_red', 'turret_blue', 
              'turret_red', 'blue_heralds', 'red_heralds'], axis=1)

**Làm 1 dictionary về dữ liệu: tướng - vị trí đi đường - vai trò bằng data_champions.csv thu thập được**

In [20]:
# Đọc dữ liệu từ tệp data_champions.csv và tạo từ điển champions_dict
data_champions = pd.read_csv('data_champions.csv')
champions_dict = {}
for index, row in data_champions.iterrows():
    champion = row['Champions']
    lane = row['Lane']
    role = row['Role']
    if champion in champions_dict:
        champions_dict[champion]['Lane'].append(lane)
        champions_dict[champion]['Role'].append(role)
    else:
        champions_dict[champion] = {'Lane': [lane], 'Role': [role]}

**Thêm các cột phân loại tướng các đường dựa trên dictionary vừa tạo**

In [21]:
# Duyệt từng hàng trong data_done.csv
for index, row in df.iterrows():
    # Duyệt từng cột tên tướng và trích xuất thông tin về lane và role tương ứng
    for column in ['top_blue', 'top_red', 'jung_blue', 'jung_red', 'mid_blue', 'mid_red', 'ad_blue', 'ad_red', 'support_blue', 'support_red']:
        champion_name = row[column]
        if champion_name not in champions_dict:
            print(f"Warning: Champion {champion_name} not found in champions_dict")
            continue
        lanes = champions_dict[champion_name]['Lane']
        roles = champions_dict[champion_name]['Role']
        lane_col = f"{column}_lane"
        role_col = f"{column}_role"
        df.at[index, lane_col] = ','.join(lanes)
        df.at[index, role_col] = ','.join(roles)

**Lưu vào file data mới**

In [22]:
df.to_csv('data_fe.csv', index=False)

**Tính điểm đội hình dựa trên lane, role: nếu có các vị trí, vai trò chính và cần thiết thì +1.**
**Nếu đội hình bị sai/ thừa/ thiếu vị trí thì điểm sẽ thấp hơn những đội hình đầy đủ**

1. Đọc file data feature engineering

In [23]:
df.to_csv('data_fe.csv', index=False)

2. Duyệt từng hàng, tính điểm cho từng đường và từng vị trí của 2 đội, sau đó cộng vào để lấy điểm tổng của cả 2 đội

In [24]:
# duyệt từng hàng của dataframe
for index, row in df.iterrows():
    score_blue = 0
    score_red = 0
    # kiểm tra lane của các vị trí của đội blue
    if row['top_blue_lane'] == 'top':
        score_blue += 1
    if row['jung_blue_lane'] == 'jungle':
        score_blue += 1
    if row['mid_blue_lane'] == 'mid':
        score_blue += 1
    if row['ad_blue_lane'] == 'adc':
        score_blue += 1
    if row['support_blue_lane'] == 'support':
        score_blue += 1
    # kiểm tra lane của các vị trí của đội red
    if row['top_red_lane'] == 'top':
        score_red += 1
    if row['jung_red_lane'] == 'jungle':
        score_red += 1
    if row['mid_red_lane'] == 'mid':
        score_red += 1
    if row['ad_red_lane'] == 'adc':
        score_red += 1
    if row['support_red_lane'] == 'support':
        score_red += 1
    df.loc[index, 'score_blue'] = score_blue
    df.loc[index, 'score_red'] = score_red

    score_blue_role = {'Marksman': 0, 'Fighter': 0, 'Tank': 0, 'Assassin': 0, 'Mage': 0, 'Support': 0}
    score_red_role = {'Marksman': 0, 'Fighter': 0, 'Tank': 0, 'Assassin': 0, 'Mage': 0, 'Support': 0}
    for position in ['top_blue_role', 'jung_blue_role', 'mid_blue_role', 'ad_blue_role', 'support_blue_role']:
        role = row[position]
        if role == 'Marksman':
            score_blue_role['Marksman'] = 1
        elif role == 'Fighter':
            score_blue_role['Fighter'] = 1
        elif role == 'Tank':
            score_blue_role['Tank'] = 1
        elif role == 'Assassin':
            score_blue_role['Assassin'] = 1
        elif role == 'Mage':
            score_blue_role['Mage'] = 1
        elif role == 'Support':
            score_blue_role['Support'] = 1
    # kiểm tra role của các vị trí của đội red
    for position in ['top_red_role', 'jung_red_role', 'mid_red_role', 'ad_red_role', 'support_red_role']:
        role = row[position]
        if role == 'Marksman':
            score_red_role['Marksman'] = 1
        elif role == 'Fighter':
            score_red_role['Fighter'] = 1
        elif role == 'Tank':
            score_red_role['Tank'] = 1
        elif role == 'Assassin':
            score_red_role['Assassin'] = 1
        elif role == 'Mage':
            score_red_role['Mage'] = 1
        elif role == 'Support':
            score_red_role['Support'] = 1
    # lưu score_blue_role, score_red_role vào 2 cột
    df.loc[index, 'score_blue_role'] = sum(score_blue_role.values())
    df.loc[index, 'score_red_role'] = sum(score_red_role.values())

df.to_csv('data_done.csv', index=False)

3. Tính chênh lệch điểm giữa 2 đội 

In [25]:
df["score_lane"] = df["score_blue"] - df["score_red"]
df["score_role"] = df["score_blue_role"] - df["score_red_role"]

4. Xóa các cột dư thừa sau khi tính chênh lệch

In [26]:
df = df.drop(['top_blue_lane', 'top_red_lane', 'jung_blue_lane', 'jung_red_lane', 'mid_blue_lane', 'mid_red_lane', 'ad_blue_lane', 
              'ad_red_lane', 'support_blue_lane', 'support_red_lane', 'top_blue_role', 'top_red_role', 'jung_blue_role', 'jung_red_role', 
              'mid_blue_role', 'mid_red_role', 'ad_blue_role', 'ad_red_role', 'support_blue_role', 'support_red_role', 'score_blue'
              , 'score_red' ,'score_blue_role' , 'score_red_role'], axis=1)

5. Lưu lại data vào data_fe.csv (data feature engineering)

In [27]:
df.to_csv("data_fe.csv", index=False)

**Tính điểm chênh lệch tỉ lệ thắng dựa vào việc lựa chọn đội hình**

1. Viết dictionary tổng trận thắng và tổng trận đấu của từng tướng/ vị trí đi đường

In [28]:
winrates = defaultdict(lambda: {position: {"total_played": 0, "total_wins": 0} for position in ["top", "jung", "mid", "ad", "support"]})

In [29]:
for position in ["top", "jung", "mid", "ad", "support"]:
    for index, row in df.iterrows():
        for team_name in ["blue", "red"]:
            position_col = f"{position}_{team_name}"            
            champions = row[position_col].split(",")
            
            for champion in champions:
                winrates[champion][position]["total_played"] += 1
                if row["team_win"] == team_name:
                    winrates[champion][position]["total_wins"] += 1

2. Tính tỉ lệ thắng = số trận thắng/ tổng số trận đấu của từng tướng/ vị trí đi đường và lưu vào file winrate.csv

In [30]:
header = ["champion", "lane", "winrate"]

# Define the file name and open it in write mode
filename = "winrates.csv"
with open(filename, mode='w', newline='') as file:

    # Create a CSV writer object and write the header
    writer = csv.writer(file)
    writer.writerow(header)

    # Loop through the `winrates` dictionary and write the data to the CSV file
    for champion, positions in winrates.items():
        for lane, data in positions.items():
            total_played = data["total_played"]
            total_wins = data["total_wins"]
            winrate = round((total_wins / total_played) * 100, 2) if total_played > 10 else 0.5
            writer.writerow([champion, lane, winrate])

3. Đọc data feature engineering

In [31]:
datadone = []
with open('data_fe.csv', mode='r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        datadone.append(row)
winrates = {}
with open('winrates.csv', mode='r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        winrates[(row['champion'], row['lane'])] = row['winrate']

4. Thay các cột vị trí đường của 2 đội từ tên tướng sang tỉ lệ thắng của tướng đó dựa vào tên tướng/lane trong dictionary

In [32]:
for row in datadone:
        row['top_blue'] = winrates.get((row['top_blue'], 'top'))
        row['top_red'] = winrates.get((row['top_red'], 'top'))
        row['jung_blue'] = winrates.get((row['jung_blue'], 'jung'))
        row['jung_red'] = winrates.get((row['jung_red'], 'jung'))
        row['mid_blue'] = winrates.get((row['mid_blue'], 'mid'))
        row['mid_red'] = winrates.get((row['mid_red'], 'mid'))
        row['ad_blue'] = winrates.get((row['ad_blue'], 'ad'))
        row['ad_red'] = winrates.get((row['ad_red'], 'ad'))
        row['support_blue'] = winrates.get((row['support_blue'], 'support'))
        row['support_red'] = winrates.get((row['support_red'], 'support'))

5. Viết thay đổi vào file data

In [33]:
with open('data_fe.csv', mode='w', newline='') as csvfile:
    fieldnames = datadone[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in datadone:
        writer.writerow(row)

6. Đọc data thay đổi

In [34]:
df = pd.read_csv('data_fe.csv')

7. Tính chênh lệch tỉ lệ thắng của 2 đội dựa trên tỉ lệ thắng của từng tướng

In [35]:
df["win_score_difference"] = df["top_blue"] + df["jung_blue"] + df["mid_blue"] + df["ad_blue"] + df["support_blue"] - df["top_red"] - df["jung_red"] - df["mid_red"] - df["ad_red"] - df["support_red"]

8. Xóa các cột dư thừa sau khi tính chênh lệch

In [36]:
df = df.drop(['top_blue','top_red', 'mid_blue', 'mid_red', 'jung_blue','jung_red','ad_blue','ad_red','support_blue','support_red'], axis=1)

9. Thay cột teamwin thành 1(blue), 0(red) rồi lưu thay đổi vào file data.csv

In [37]:
df['team_win'] = df['team_win'].replace({'blue': 1, 'red': 0})
df.to_csv("data_fe.csv", index=False)

**Chia 10k dữ liệu thành 10 file, mỗi file 1k**

In [38]:
for i in range(10):
    start_index = i * 1000
    end_index = start_index + 1000
    df_subset = df.iloc[start_index:end_index, :]
    filename = f"data_fe_1000_part_{i+1}.csv"
    df_subset.to_csv(filename, index=False)